# Run FoldX Stability Analysis

In [27]:
import pandas as pd
import subprocess
import shutil

In [ ]:
## Load the experiments data
experiments = pd.read_excel('../../../experiments.xlsx', sheet_name='HA1-docking')

## Filter down to diffused antbodies
experiments = experiments[experiments['antibody_type'] == 'diffused']

In [ ]:
## Make blank dataframe for FoldX results
foldx_cols = [
    'antibody_id',
    'Pdb', 'Total Energy', 'Backbone Hbond', 
    'Sidechain Hbond', 'Van der Waals', 'Electrostatics',
    'Solvation Polar', 'Solvation Hydrophobic', 'Van der Waals clashes',
    'Entropy Side Chain', 'Entropy Main Chain', 'Sloop Entropy',
    'Mloop Entropy', 'Cis Bond', 'Torsional Clash', 'Backbone Clash',
    'Helix Dipole', 'Water Bridge', 'Disulfide', 'Electrostatic Kon',
    'Partial Covalent Bonds', 'Energy Ionisation',
    'Entropy Complex', 'Residue Number'
]

foldx_df = pd.DataFrame(columns=foldx_cols)

In [ ]:
## Loop through each experiment
for index, row in experiments.iterrows():
    antibody_id = row['antibody_id']
    best_model_name = row['best_model_name'].strip()

    ## Copy the best model PDB file to the current directory
    shutil.copyfile(f'../../../../experiments/{antibody_id}/4_docking/output/08_mdscoring/{best_model_name}', f'./{best_model_name}')

    
    ## Construct the command to run the script
    command = [
        '../executables/foldx5_1Mac/foldx_20251231',
        '--command=Stability',
        f'--pdb={best_model_name}'
    ]
    
    ## Execute the command
    subprocess.run(command, check=True)

    ## Read the output file and append to the DataFrame
    output_file = f'{best_model_name.replace('.pdb', '')}_0_ST.fxout'

    row_df = pd.read_csv(output_file, sep='\t', header=None, names=foldx_cols)
    row_df['antibody_id'] = antibody_id
    foldx_df = pd.concat([foldx_df, row_df], ignore_index=True)

## Save the results to a CSV file
foldx_df.to_csv('foldx_results.csv', index=False)


In [ ]:
foldx_df

,antibody_id,Pdb,Total Energy,Backbone Hbond,Sidechain Hbond,Van der Waals,Electrostatics,Solvation Polar,Solvation Hydrophobic,Van der Waals clashes,...,Torsional Clash,Backbone Clash,Helix Dipole,Water Bridge,Disulfide,Electrostatic Kon,Partial Covalent Bonds,Energy Ionisation,Entropy Complex,Residue Number
0,kinetic-template,380.928,-248.952,-129.209,-538.534,-19.70830,783.936,-677.015,65.8179,277.717,...,154.906,-0.814925,0,-20.0083,0.021796,0,1.63670,0,1532,NaN
1,partial-lagoon,327.966,-282.228,-141.631,-567.592,-25.55330,815.996,-719.227,61.1246,296.721,...,157.860,-1.418250,0,-15.5556,-0.472836,0,1.18062,0,1532,NaN
2,glowing-avocet,387.708,-266.897,-127.787,-546.993,-12.42840,790.280,-691.679,62.4911,280.121,...,151.663,-1.336130,0,-15.5744,-0.097528,0,1.81122,0,1532,NaN
3,approximate-entrepreneur,368.221,-258.827,-123.948,-538.458,-19.49520,783.753,-681.279,65.8194,277.294,...,154.819,-1.299420,0,-15.5654,-0.547132,0,1.62443,0,1532,NaN
4,recursive-basin,380.530,-263.734,-133.146,-550.561,-21.81540,807.295,-695.531,67.0768,285.470,...,160.539,-1.259960,0,-20.0305,0.045079,0,2.04628,0,1532,NaN
5,free-hearth,363.952,-269.994,-146.584,-563.673,-17.74650,809.028,-717.969,78.3402,288.975,...,147.661,-1.398580,0,-20.0066,-0.043177,0,2.15741,0,1532,NaN
6,smoky-latitude,365.573,-272.596,-149.815,-554.222,-21.42790,810.172,-693.394,69.6268,290.117,...,155.313,-0.781651,0,-24.4266,-0.413471,0,1.91473,0,1532,NaN
7,internal-rundown,407.932,-246.936,-128.219,-544.973,-12.99300,794.407,-686.863,66.8028,277.212,...,158.526,-0.903698,0,-20.0471,-0.006198,0,1.52780,0,1532,NaN
8,critical-tin,334.474,-278.063,-140.366,-562.626,-21.98820,813.438,-708.647,60.5255,286.125,...,167.813,-1.321030,0,-19.9860,-0.797153,0,1.73762,0,1532,NaN
9,gilded-stud,338.174,-280.839,-147.811,-550.654,-19.00560,804.701,-684.272,58.9074,281.424,...,157.016,-1.374370,0,-20.0102,-0.594988,0,1.66580,0,1532,NaN


In [32]:
foldx_df.to_csv('foldx_results.csv', index=False)

In [ ]:
## Clean up the output files
subprocess.run(['rm', '*.fxout'], check=True)
subprocess.run(['rm', '*.pdb'], check=True)